In [4]:
import pandas as pd
import os
import glob
import ants
from helpers import *
import gc

In [11]:
adni_sample_root = '/media/leelabsg-storage1/DATA/ADNI/adni_t1s_baseline/sub-037S6222X20180530'
adni_sample_brain = adni_sample_root + '/brain.nii.gz'
adni_sample_mni = adni_sample_root + '/brain_to_MNI_nonlin.nii.gz'

ants_adni_brain = ants.image_read(adni_sample_brain)
ants_adni_mni = ants.image_read(adni_sample_mni)

In [12]:
explore_3D_array(ants_adni_brain)

interactive(children=(IntSlider(value=127, description='SLICE', max=255), Output()), _dom_classes=('widget-int…

In [13]:
explore_3D_array(ants_adni_mni)

interactive(children=(IntSlider(value=45, description='SLICE', max=90), Output()), _dom_classes=('widget-inter…

In [5]:
# lab storage 내 UKB Brain MRI data 기본 경로
root = '/media/leelabsg-storage1/DATA/UKBB/bulk/20252_uz'

# Dataset size
data = glob.glob(f'{root}/*')

In [7]:
len(data)

41184

In [58]:
subjects = list()

In [59]:
for i in range(len(data)):
    subject = data[i].split('/')[-1].split('_')[0]
    subjects.append(int(subject))

In [60]:
len(subjects)

41184

In [35]:
#Download the list of IDs
ii=np.load('/media/leelabsg-storage1/DATA/UKBB/brain_age/ID_128_full.npy',allow_pickle=True)
a=ii.tolist()
ii_pd=pd.DataFrame(a)
cap=ii_pd.rename(columns={0:'id', 1:'age'})

In [36]:
# Disease metadata
disease = pd.read_csv('/media/leelabsg-storage1/DATA/UKBB/brain_age/PEDMASTER_ALL_20180514.txt', sep='\t')
disease

,FID,Sex,birthYear,BATCH,genotyping.array,Batch,Plate.Name,Well,Cluster.CR,dQC,...,X1007,X1008,X1009,X1010,X1011,X1012,X1013,X1014,X1015,X1019
0,1000012,M,1956,Batch_b061,UKBB,Batch_b061,SMP4_0014446A,G12,99.801,0.98000,...,0,0,0,0,0,0,0,0,0,0
1,1000029,M,1938,UKBiLEVEAX_b2,UKBL,UKBiLEVEAX_b2,SMP4_0008467,H05,99.691,0.98722,...,0,0,0,0,0,0,0,0,0,0
2,1000031,M,1944,Batch_b094,UKBB,Batch_b094,SMP4_0009989B,B06,99.849,0.97000,...,0,0,0,0,0,0,0,0,0,0
3,1000047,M,1949,UKBiLEVEAX_b9,UKBL,UKBiLEVEAX_b9,SMP4_0008702,G01,99.346,0.99263,...,0,0,0,0,0,0,0,0,0,0
4,1000050,F,1952,Batch_b014,UKBB,Batch_b014,SMP4_0010350A,A10,98.802,0.96000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487390,6026297,F,1944,Batch_b080,UKBB,Batch_b080,SMP4_0006101A,F03,99.557,0.99000,...,0,0,0,0,0,0,0,0,0,0
487391,6026308,F,1950,Batch_b065,UKBB,Batch_b065,SMP4_0014916A,E03,99.674,0.97000,...,0,0,0,0,0,0,0,0,0,0
487392,6026314,F,1961,Batch_b012,UKBB,Batch_b012,SMP4_0010349A,C09,99.188,0.97000,...,0,0,0,0,0,0,0,0,0,0
487393,6026320,M,1947,Batch_b023,UKBB,Batch_b023,SMP4_0015148A,A10,99.671,0.96000,...,0,0,0,0,0,0,0,0,0,0


In [37]:
mapping = pd.read_csv('/media/leelabsg-storage1/DATA/UKBB/brain_age/mapping.csv')
mapping_mri = mapping[mapping['Shawn'].isin(cap['id'].copy())]  #37792

In [38]:
print(disease.shape)
print(cap.shape)
print(mapping_mri.shape)

(487395, 1756)
(35914, 2)
(34186, 2)


In [39]:
mapping_mri.columns = ['FID', 'Shawn'] 
mri_disease = pd.merge(disease, mapping_mri, on='FID') #37724
print(mri_disease.shape)

(487395, 1756)


In [41]:
mri_disease

,FID,Sex,birthYear,BATCH,genotyping.array,Batch,Plate.Name,Well,Cluster.CR,dQC,...,X1008,X1009,X1010,X1011,X1012,X1013,X1014,X1015,X1019,Shawn
0,1000119,F,1947,Batch_b070,UKBB,Batch_b070,SMP4_0013032A,H09,99.724,0.99,...,0,0,0,0,0,0,0,0,0,3187611
1,1000158,M,1948,Batch_b031,UKBB,Batch_b031,SMP4_0012819A,E10,98.864,0.97,...,0,1,0,0,0,0,0,0,0,5104403
2,1000295,F,1948,Batch_b012,UKBB,Batch_b012,SMP4_0010824A,E06,99.261,0.98,...,0,0,0,0,0,0,0,0,0,1692267
3,1000615,F,1943,Batch_b002,UKBB,Batch_b002,SMP4_0012742A,A11,99.690,0.99,...,0,0,0,0,0,0,0,0,0,3692362
4,1000898,M,1948,Batch_b093,UKBB,Batch_b093,SMP4_0011464A,D04,99.364,0.99,...,0,0,0,0,0,0,0,0,0,4457188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34124,6024550,F,1950,Batch_b087,UKBB,Batch_b087,SMP4_0017451A,D08,98.756,0.97,...,0,0,0,0,0,0,0,0,0,3641054
34125,6025217,F,1954,Batch_b009,UKBB,Batch_b009,SMP4_0016579A,B01,99.746,0.99,...,0,0,0,0,0,0,0,0,0,3674565
34126,6025449,F,1946,Batch_b007,UKBB,Batch_b007,SMP4_0016853A,E05,99.289,0.95,...,0,0,0,0,0,0,0,0,0,3793096
34127,6025579,M,1944,Batch_b035,UKBB,Batch_b035,SMP4_0014292A,E11,98.459,0.95,...,0,0,0,0,0,0,0,0,1,5760839


In [67]:
disease_mri = set(mri_disease['FID'].tolist())
len(disease_mri)

34129

In [66]:
diff = disease_mri - set(subjects)
len(diff)

34129

In [65]:
len(diff)

34129

In [62]:
type(mri_disease['FID'].tolist()[0])

int

In [61]:
type(subjects[0])

int